In [3]:
# Challenge Foundation Setup
# Run this cell FIRST before any challenge code

import subprocess
import os
import time
import math
import random
from IPython.display import display, HTML

def run_ros2_command(cmd, cwd="/home/jovyan", background=False):
    """Execute ROS2 commands and display output in notebook - TESTED VERSION"""
    
    # Complete environment setup for ROS2 in container
    env = os.environ.copy()
    env.update({
        'ROS_DOMAIN_ID': os.environ.get('ROS_DOMAIN_ID', '0'),
        'RMW_IMPLEMENTATION': os.environ.get('RMW_IMPLEMENTATION', 'rmw_fastrtps_cpp'),
        'DISPLAY': ':1',
        'ROS_DISTRO': 'jazzy',
        'AMENT_PREFIX_PATH': '/opt/ros/jazzy',
        'LD_LIBRARY_PATH': '/opt/ros/jazzy/lib:/opt/ros/jazzy/lib/x86_64-linux-gnu',
        'PATH': '/opt/ros/jazzy/bin:' + env.get('PATH', ''),
        'PYTHONPATH': '/opt/ros/jazzy/lib/python3.12/site-packages:' + env.get('PYTHONPATH', ''),
        'CMAKE_PREFIX_PATH': '/opt/ros/jazzy',
        'PKG_CONFIG_PATH': '/opt/ros/jazzy/lib/pkgconfig',
        'COLCON_PREFIX_PATH': '/opt/ros/jazzy'
    })
    
    # Use bash explicitly and source setup files
    bash_cmd = f"""
    bash -c '
    source /opt/ros/jazzy/setup.bash 2>/dev/null || true
    export ROS_DOMAIN_ID={env["ROS_DOMAIN_ID"]}
    export RMW_IMPLEMENTATION={env["RMW_IMPLEMENTATION"]}
    {cmd}
    '
    """
    
    try:
        if background:
            # Run in background for continuous commands
            process = subprocess.Popen(bash_cmd, shell=True, cwd=cwd, env=env)
            print(f"Background command started: {cmd}")
            return process
        else:
            result = subprocess.run(
                bash_cmd,
                shell=True, capture_output=True, text=True, cwd=cwd, env=env, timeout=10
            )
            
            print(f"Command: {cmd}")
            if result.stdout.strip():
                print(f"Output:\n{result.stdout}")
            if result.stderr.strip() and not "QStandardPaths" in result.stderr:
                print(f"Errors:\n{result.stderr}")
            return result
    except subprocess.TimeoutExpired:
        print(f"Command timed out: {cmd}")
        return None
    except Exception as e:
        print(f"Command failed: {e}")
        return None

def test_ros2_connection():
    """Test if ROS2 is working properly - ESSENTIAL CHECK"""
    print("🔍 Testing ROS2 connection...")
    
    # Test 1: Check if ros2 command works
    result1 = run_ros2_command("ros2 --help")
    
    # Test 2: List topics
    print("\n--- Available Topics ---")
    result2 = run_ros2_command("ros2 topic list")
    
    # Test 3: Check if turtlesim is running
    print("\n--- Active Nodes ---")
    result3 = run_ros2_command("ros2 node list")
    
    # Quick analysis
    if result2 and "/turtle1/cmd_vel" in result2.stdout:
        print("✅ TurtleSim is running and ready!")
    else:
        print("⚠️  TurtleSim may not be running. Start it in the desktop terminal:")
        print("   ros2 run turtlesim turtlesim_node")
    
    return result1, result2, result3

def quick_turtle_test():
    """Quick test to move turtle and verify everything works"""
    print("🐢 Quick turtle movement test...")
    
    # Move forward briefly
    result = run_ros2_command('ros2 topic pub --once /turtle1/cmd_vel geometry_msgs/msg/Twist "{linear: {x: 1.0, y: 0.0, z: 0.0}, angular: {x: 0.0, y: 0.0, z: 0.0}}"')
    
    if result and result.returncode == 0:
        print("✅ Turtle movement command successful!")
        print("🎯 Check your desktop window - turtle should have moved!")
    else:
        print("❌ Turtle movement failed. Check if TurtleSim is running.")
    
    return result

def display_challenge_header(challenge_num, title, learning_objectives):
    """Display formatted challenge header"""
    header_html = f"""
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 25px; border-radius: 15px; margin: 20px 0;">
        <h1 style="margin: 0 0 15px 0;">🎯 Challenge {challenge_num}: {title}</h1>
        <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 8px;">
            <h3 style="margin: 0 0 10px 0;">📚 Learning Objectives:</h3>
            <ul style="margin: 0; padding-left: 20px;">
    """
    
    for objective in learning_objectives:
        header_html += f"<li>{objective}</li>"
    
    header_html += """
            </ul>
        </div>
    </div>
    """
    
    display(HTML(header_html))

def mark_task_complete(task_name):
    """Mark a task as completed with visual feedback"""
    complete_html = f"""
    <div style="background: #d4edda; color: #155724; padding: 10px; border-radius: 8px; border-left: 4px solid #28a745; margin: 10px 0;">
        <strong>✅ COMPLETED:</strong> {task_name}
    </div>
    """
    display(HTML(complete_html))

def show_hint(hint_text):
    """Display a hint in a styled box"""
    hint_html = f"""
    <div style="background: #fff3cd; color: #856404; padding: 15px; border-radius: 8px; border-left: 4px solid #ffc107; margin: 10px 0;">
        <strong>💡 HINT:</strong> {hint_text}
    </div>
    """
    display(HTML(hint_html))

def show_debug_info():
    """Show debugging information for troubleshooting"""
    debug_html = """
    <div style="background: #f8d7da; color: #721c24; padding: 15px; border-radius: 8px; border-left: 4px solid #dc3545; margin: 10px 0;">
        <strong>🔧 DEBUG MODE:</strong> Use this when things aren't working
    </div>
    """
    display(HTML(debug_html))
    
    print("Environment variables:")
    print(f"ROS_DOMAIN_ID: {os.environ.get('ROS_DOMAIN_ID', 'Not set')}")
    print(f"RMW_IMPLEMENTATION: {os.environ.get('RMW_IMPLEMENTATION', 'Not set')}")
    print(f"DISPLAY: {os.environ.get('DISPLAY', 'Not set')}")
    
    print("\nROS2 system check:")
    run_ros2_command("ros2 doctor --report")

# Essential utility functions for challenges
def parse_pose_output(pose_text):
    """Parse turtle pose from ros2 topic echo output"""
    try:
        lines = pose_text.strip().split('\n')
        x, y, theta = None, None, None
        
        for line in lines:
            if 'x:' in line:
                x = float(line.split(':')[1].strip())
            elif 'y:' in line:
                y = float(line.split(':')[1].strip())  
            elif 'theta:' in line:
                theta = float(line.split(':')[1].strip())
        
        return {'x': x, 'y': y, 'theta': theta}
    except:
        return {'x': 0, 'y': 0, 'theta': 0}

def degrees_to_radians(degrees):
    """Convert degrees to radians"""
    return degrees * math.pi / 180

def radians_to_degrees(radians):
    """Convert radians to degrees"""
    return radians * 180 / math.pi

print("🚀 Foundation code loaded successfully!")
print("📋 Available functions:")
print("  • run_ros2_command(cmd) - Execute ROS2 commands")
print("  • test_ros2_connection() - Check ROS2 setup")  
print("  • quick_turtle_test() - Test turtle movement")
print("  • display_challenge_header() - Format challenge display")
print("  • mark_task_complete() - Mark tasks done")
print("  • show_hint() - Display helpful hints")
print("  • show_debug_info() - Troubleshooting info")
print("\n🔧 Next step: Run test_ros2_connection() to verify setup")

🚀 Foundation code loaded successfully!
📋 Available functions:
  • run_ros2_command(cmd) - Execute ROS2 commands
  • test_ros2_connection() - Check ROS2 setup
  • quick_turtle_test() - Test turtle movement
  • display_challenge_header() - Format challenge display
  • mark_task_complete() - Mark tasks done
  • show_hint() - Display helpful hints
  • show_debug_info() - Troubleshooting info

🔧 Next step: Run test_ros2_connection() to verify setup
